In [1]:
import os
import pandas as pd
import geopandas as gpd
import tarfile
import requests
import numpy as np


In [2]:


# --------------------------------------------------------
# 0. Config
# --------------------------------------------------------
# Base URL for TLC trip record data (Yellow Taxi, 2023, Parquet)
BASE_URL = "https://d37ci6vzurychx.cloudfront.net/trip-data"

# List of all 12 months in 2023
months = range(1, 13)

# Target save path
OUTPUT_PATH = "../data/raw/nyc_taxi_2023.csv"

# --------------------------------------------------------
# 1. Download + Load all 12 Parquet files into a single DataFrame
# --------------------------------------------------------
dfs = []

for month in months:
    file_url = f"{BASE_URL}/yellow_tripdata_2023-{month:02d}.parquet"
    print(f"Loading {file_url} ...")

    # NOTE: Needs pyarrow or fastparquet installed:
    #   pip install pyarrow
    df_month = pd.read_parquet(file_url)
    dfs.append(df_month)

    print(df_month,"fd")
    print(dfs,"vij")
# Concatenate all monthly data into one year-long DataFrame
taxi_2023 = pd.concat(dfs, ignore_index=True)

# --------------------------------------------------------
# 2. Basic info: shape, columns, dtypes
# --------------------------------------------------------
print("\n=== DataFrame Shape ===")
print(taxi_2023.shape)  # (rows, columns)

print("\n=== Column Names and Data Types ===")
print(taxi_2023.dtypes)

# --------------------------------------------------------
# 3. First 5 rows
# --------------------------------------------------------
print("\n=== First 5 Rows ===")
print(taxi_2023.head())

# --------------------------------------------------------
# 4. Missing value counts
# --------------------------------------------------------
print("\n=== Missing Values per Column ===")
missing_counts = taxi_2023.isna().sum()
print(missing_counts)

# --------------------------------------------------------
# 5. Convert datetime columns and check dtypes
# --------------------------------------------------------
datetime_cols = ["tpep_pickup_datetime", "tpep_dropoff_datetime"]

for col in datetime_cols:
    if col in taxi_2023.columns:
        taxi_2023[col] = pd.to_datetime(taxi_2023[col], errors="coerce")

print("\n=== Data Types After Datetime Conversion ===")
print(taxi_2023.dtypes)

# --------------------------------------------------------
# 6. Summary statistics for key columns
# --------------------------------------------------------
key_numeric_cols = ["fare_amount", "trip_distance", "PULocationID", "DOLocationID"]
existing_numeric_cols = [c for c in key_numeric_cols if c in taxi_2023.columns]

print("\n=== Summary Statistics for Key Numeric Columns ===")
print(taxi_2023[existing_numeric_cols].describe(include="all"))

if "tpep_pickup_datetime" in taxi_2023.columns:
    print("\n=== Pickup Datetime Range ===")
    print("Min:", taxi_2023["tpep_pickup_datetime"].min())
    print("Max:", taxi_2023["tpep_pickup_datetime"].max())

# --------------------------------------------------------
# 7. Total number of records
# --------------------------------------------------------
total_records = len(taxi_2023)
print("\n=== Total Number of Records ===")
print(total_records)

# --------------------------------------------------------
# 8. Save to CSV for later use
# --------------------------------------------------------
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
taxi_2023.to_csv(OUTPUT_PATH, index=False)

print(f"\n DataFrame successfully saved to: {OUTPUT_PATH}")


Loading https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet ...
         VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0               2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1               2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2               2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3               1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4               2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   
...           ...                  ...                   ...              ...   
3066761         2  2023-01-31 23:58:34   2023-02-01 00:12:33              NaN   
3066762         2  2023-01-31 23:31:09   2023-01-31 23:50:36              NaN   
3066763         2  2023-01-31 23:01:05   2023-01-31 23:25:36              NaN   
3066764         2  2023-01-31 23:40:00   2023-01-31 23:53:00              NaN   
3066765         2

In [3]:


# ------------------------------------------------------------------
# 0. Paths & URLs
# ------------------------------------------------------------------
DATASET_PAGE = "https://catalog.data.gov/dataset/nyc-taxi-zones-131e4/resource/2dd4a8a3-bf0b-46d4-b11a-0b5ce833527c"
SHAPE_ZIP_URL = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip"

print("Dataset catalog page (for reference):", DATASET_PAGE)
print("Reading shapefile directly from:", SHAPE_ZIP_URL)

RAW_DIR = "../data/raw"
os.makedirs(RAW_DIR, exist_ok=True)

csv_output_path = os.path.join(RAW_DIR, "nyc_taxi_zones.csv")
geojson_output_path = os.path.join(RAW_DIR, "nyc_taxi_zones.geojson")
pickle_output_path = os.path.join(RAW_DIR, "nyc_taxi_zones.pkl")

# ------------------------------------------------------------------
# 1. Load geographic data directly from the zipped shapefile
# ------------------------------------------------------------------
gdf = gpd.read_file(SHAPE_ZIP_URL)

# ------------------------------------------------------------------
# 2. Structure: columns, first rows, dtypes, total zones
# ------------------------------------------------------------------
print("\n=== Columns ===")
print(list(gdf.columns))

print("\n=== First 10 Rows ===")
print(gdf.head(10))

print("\n=== Data Types ===")
print(gdf.dtypes)

total_zones = len(gdf)
print(f"\n=== Total Number of Zones ===\n{total_zones}")

# ------------------------------------------------------------------
# 3. Verify key information
# ------------------------------------------------------------------
# Normalize column names for convenience
cols_lower = {c.lower(): c for c in gdf.columns}
loc_col = cols_lower.get("locationid", "LocationID")
borough_col = cols_lower.get("borough", "Borough")
zone_col = cols_lower.get("zone", "Zone")

# Unique boroughs
unique_boroughs = gdf[borough_col].unique()
print("\n=== Unique Boroughs ===")
print(unique_boroughs)

# Zones per borough
zones_per_borough = gdf.groupby(borough_col).size().sort_values(ascending=False)
print("\n=== Zone Counts per Borough ===")
print(zones_per_borough)

# Sample zones (up to 3) from each borough
print("\n=== Sample Zones from Each Borough (up to 3 each) ===")
sample_zones = (
    gdf.groupby(borough_col)
       .head(3)[[loc_col, borough_col, zone_col]]
       .sort_values([borough_col, loc_col])
)
print(sample_zones)

# ------------------------------------------------------------------
# 4. Save to CSV (no geometry) and GeoJSON / Pickle
# ------------------------------------------------------------------
# CSV: drop geometry
non_geom_cols = [c for c in gdf.columns if c.lower() != "geometry"]
gdf[non_geom_cols].to_csv(csv_output_path, index=False)
print(f"\n CSV (without geometry) saved to: {csv_output_path}")

# GeoJSON: full geometry for mapping
gdf.to_file(geojson_output_path, driver="GeoJSON")
print(f" GeoJSON (with geometry) saved to: {geojson_output_path}")

gdf.to_pickle(pickle_output_path)
print(f" Pickle saved to: {pickle_output_path}")

# ------------------------------------------------------------------
# 5. Summary
# ------------------------------------------------------------------
print("\n=== Summary ===")
print(f"Total number of zones: {total_zones}")
print("\nZone counts per borough:")
print(zones_per_borough)
print("\nSample LocationID / Borough / Zone:")
print(sample_zones.head())

print("\n Taxi zones geographic data successfully loaded and saved.")


Dataset catalog page (for reference): https://catalog.data.gov/dataset/nyc-taxi-zones-131e4/resource/2dd4a8a3-bf0b-46d4-b11a-0b5ce833527c
Reading shapefile directly from: https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip

=== Columns ===
['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'LocationID', 'borough', 'geometry']

=== First 10 Rows ===
   OBJECTID  Shape_Leng  Shape_Area                     zone  LocationID  \
0         1    0.116357    0.000782           Newark Airport           1   
1         2    0.433470    0.004866              Jamaica Bay           2   
2         3    0.084341    0.000314  Allerton/Pelham Gardens           3   
3         4    0.043567    0.000112            Alphabet City           4   
4         5    0.092146    0.000498            Arden Heights           5   
5         6    0.150491    0.000606  Arrochar/Fort Wadsworth           6   
6         7    0.107417    0.000390                  Astoria           7   
7         8    0.027591    0.000027  

In [4]:


# ------------------------------------------------------------------
# 0. Config / Paths
# ------------------------------------------------------------------
GSOD_ARCHIVE_BASE = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/archive/"
YEAR = 2023
GSOD_YEAR_URL = f"{GSOD_ARCHIVE_BASE}{YEAR}.tar.gz"

RAW_DIR = "../data/raw"
os.makedirs(RAW_DIR, exist_ok=True)

tar_path = os.path.join(RAW_DIR, f"gsod_{YEAR}.tar.gz")
OUTPUT_PATH = os.path.join(RAW_DIR, "weather_2023_nyc.csv")

# Station name keywords for NYC airports
NYC_STATION_KEYWORDS = [
    "JFK INTERNATIONAL AIRPORT",
    "LAGUARDIA AIRPORT",
    "NEWARK LIBERTY INTERNATIONAL AIRPORT",
]

print(f"Downloading GSOD archive for {YEAR} from:\n{GSOD_YEAR_URL}")

# ------------------------------------------------------------------
# 1. Download yearly GSOD archive
# ------------------------------------------------------------------
resp = requests.get(GSOD_YEAR_URL, stream=True)
resp.raise_for_status()

with open(tar_path, "wb") as f:
    for chunk in resp.iter_content(chunk_size=1024 * 1024):
        if chunk:
            f.write(chunk)

print(f" Downloaded yearly GSOD archive to: {tar_path}")

# ------------------------------------------------------------------
# 2. Extract NYC stations from the tar.gz
# ------------------------------------------------------------------
nyc_dfs = []

with tarfile.open(tar_path, "r:gz") as tf:
    members = [m for m in tf.getmembers() if m.isfile() and m.name.endswith(".csv")]
    print(f"Archive contains {len(members)} station CSV files for {YEAR}.")

    for member in members:
        file_obj = tf.extractfile(member)
        if file_obj is None:
            continue

        tmp = pd.read_csv(file_obj)

        if "NAME" not in tmp.columns:
            continue

        station_name = str(tmp["NAME"].iloc[0])

        if any(keyword in station_name.upper() for keyword in NYC_STATION_KEYWORDS):
            print(f" Found NYC station in file: {member.name}  →  {station_name}")
            nyc_dfs.append(tmp[["STATION", "NAME", "DATE", "TEMP", "PRCP"]])

if not nyc_dfs:
    raise RuntimeError("No NYC airport stations found in GSOD archive. Check keywords or year.")

nyc_weather_raw = pd.concat(nyc_dfs, ignore_index=True)
print(f"\nCombined NYC raw weather shape: {nyc_weather_raw.shape}")

# ------------------------------------------------------------------
# 3. Parse dates, filter to 2023, keep relevant columns
# ------------------------------------------------------------------

nyc_weather_raw["DATE"] = pd.to_datetime(nyc_weather_raw["DATE"], errors="coerce")

nyc_weather_raw = nyc_weather_raw[
    (nyc_weather_raw["DATE"].dt.year == YEAR)
].copy()

print(f"After filtering to {YEAR}, shape: {nyc_weather_raw.shape}")

# ------------------------------------------------------------------
# 4. Data cleaning and unit handling
# ------------------------------------------------------------------

nyc_weather_raw["TEMP"] = pd.to_numeric(nyc_weather_raw["TEMP"], errors="coerce")
nyc_weather_raw["PRCP"] = pd.to_numeric(nyc_weather_raw["PRCP"], errors="coerce")

for col in ["TEMP", "PRCP"]:
    nyc_weather_raw.loc[nyc_weather_raw[col] >= 9999, col] = np.nan

temperature_f = nyc_weather_raw["TEMP"]

precip_in = nyc_weather_raw["PRCP"]

# ------------------------------------------------------------------
# 5. Aggregate across stations (JFK, LGA, EWR) by DATE
# ------------------------------------------------------------------
agg_weather = (
    nyc_weather_raw
    .groupby("DATE", as_index=False)
    .agg({
        "TEMP": "mean",
        "PRCP": "mean"
    })
)

agg_weather = agg_weather.rename(columns={
    "DATE": "date",
    "TEMP": "temperature_avg",
    "PRCP": "precipitation",
})

agg_weather["date"] = agg_weather["date"].dt.date

# ------------------------------------------------------------------
# 6. Derived columns: is_raining, temp_category
# ------------------------------------------------------------------
agg_weather["is_raining"] = (agg_weather["precipitation"] > 0).astype(int)

def temp_category(temp_f):
    if pd.isna(temp_f):
        return np.nan
    if temp_f < 32:
        return "cold"
    elif temp_f > 75:
        return "hot"
    else:
        return "normal"

agg_weather["temp_category"] = agg_weather["temperature_avg"].apply(temp_category)

# ------------------------------------------------------------------
# 7. Display requested summaries
# ------------------------------------------------------------------
print("\n=== Date Range Covered ===")
print("Start:", agg_weather["date"].min())
print("End  :", agg_weather["date"].max())

print("\n=== Number of Records (days) ===")
print(len(agg_weather))

print("\n=== Missing Value Summary ===")
print(agg_weather.isna().sum())

print("\n=== Temperature Statistics (°F) ===")
print(agg_weather["temperature_avg"].describe())

print("\n=== Precipitation Statistics (inches) ===")
print(agg_weather["precipitation"].describe())

rainy_days = agg_weather["is_raining"].sum()
print(f"\n=== Rainy Days Count (PRCP > 0) ===\n{rainy_days}")

print("\n=== Sample 10 Rows ===")
print(agg_weather.head(10))

# ------------------------------------------------------------------
# 8. Save to CSV in required schema
# ------------------------------------------------------------------
agg_weather = agg_weather[[
    "date",
    "temperature_avg",
    "precipitation",
    "is_raining",
    "temp_category",
]]

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
agg_weather.to_csv(OUTPUT_PATH, index=False)

print(f"\n Weather data for NYC {YEAR} saved to:\n{os.path.abspath(OUTPUT_PATH)}")


https://www.ncei.noaa.gov/data/global-summary-of-the-day/archive/2023.tar.gz
 Downloaded yearly GSOD archive to: ../data/raw/gsod_2023.tar.gz
Archive contains 12311 station CSV files for 2023.
 Found NYC station in file: 72502014734.csv  →  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US
 Found NYC station in file: 72503014732.csv  →  LAGUARDIA AIRPORT, NY US
 Found NYC station in file: 74486094789.csv  →  JFK INTERNATIONAL AIRPORT, NY US

Combined NYC raw weather shape: (1095, 5)
After filtering to 2023, shape: (1095, 5)

=== Date Range Covered ===
Start: 2023-01-01
End  : 2023-12-31

=== Number of Records (days) ===
365

=== Missing Value Summary ===
date               0
temperature_avg    0
precipitation      0
is_raining         0
temp_category      0
dtype: int64

=== Temperature Statistics (°F) ===
count    365.000000
mean      57.734886
std       14.515441
min       13.166667
25%       45.200000
50%       56.600000
75%       71.266667
max       85.266667
Name: temperature_avg, dtype